In [1]:
import os
import itertools
import matplotlib as mpl


def set_size(width, fraction=1):
    """Set figure dimensions to avoid scaling in LaTeX.

    From: https://jwalton.info/Embed-Publication-Matplotlib-Latex/

    Parameters
    ----------
    width: float
            Document textwidth or columnwidth in pts
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**0.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim


width, height = set_size(252, fraction=1.0)
widht = 2 * width

# width = 5
# height = width/1.618
fontsize = 9

mpl.rcParams.update(
    {
        "font.size": fontsize,
        "figure.figsize": (width, height),
        "figure.facecolor": "white",
        "savefig.dpi": 360,
        "figure.subplot.bottom": 0.125,
        "figure.edgecolor": "white",
        "xtick.labelsize": fontsize,
        "ytick.labelsize": fontsize,
    }
)

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from deephyper.skopt.moo import pareto_front, non_dominated_set, hypervolume
from deephyper.skopt.utils import cook_objective_scaler
from deephyper.search.hps import CBO

from deephyper.search.hps._mpi_doptuna import MPIDistributedOptuna

In [2]:
# task_name = "navalpropulsion"
# task_name = "parkinsonstelemonitoring"
# task_name = "proteinstructure"
task_name = "slicelocalization"
os.environ["DEEPHYPER_BENCHMARK_TASK"] = task_name
os.environ["DEEPHYPER_BENCHMARK_MOO"] = "1"

objective_columns = ["objective_0", "objective_1"]

repetitions = 10

max_evals = 200

In [3]:
import deephyper_benchmark as dhb

dhb.load("HPOBench/tabular")
from deephyper_benchmark.lib.hpobench.tabular import hpo

print(f"{hpo.DEEPHYPER_BENCHMARK_TASK=}")
print(f"{hpo.DEEPHYPER_BENCHMARK_MOO=}")

hpo.problem

hpo.DEEPHYPER_BENCHMARK_TASK='slicelocalization'
hpo.DEEPHYPER_BENCHMARK_MOO=True


Configuration space object:
  Hyperparameters:
    activation_fn_1, Type: Categorical, Choices: {tanh, relu}, Default: tanh
    activation_fn_2, Type: Categorical, Choices: {tanh, relu}, Default: tanh
    batch_size, Type: Ordinal, Sequence: {8, 16, 32, 64}, Default: 8
    dropout_1, Type: Ordinal, Sequence: {0.0, 0.3, 0.6}, Default: 0.0
    dropout_2, Type: Ordinal, Sequence: {0.0, 0.3, 0.6}, Default: 0.0
    init_lr, Type: Ordinal, Sequence: {0.0005, 0.001, 0.005, 0.01, 0.05, 0.1}, Default: 0.0005
    lr_schedule, Type: Categorical, Choices: {cosine, const}, Default: cosine
    n_units_1, Type: Ordinal, Sequence: {16, 32, 64, 128, 256, 512}, Default: 16
    n_units_2, Type: Ordinal, Sequence: {16, 32, 64, 128, 256, 512}, Default: 16

In [4]:
scalers = ["identity", "minmaxlog", "log", "quantile-uniform", "minmax"]
strategies = ["Linear", "Chebyshev", "PBI"]

for i in range(repetitions):
    print(f"Starting round {i}...")

    # # Random
    # print("Random search...")
    # results = CBO(
    #     hpo.problem,
    #     hpo.run,
    #     surrogate_model="DUMMY",
    #     objective_scaler="identity",
    #     moo_scalarization_strategy="Linear",
    #     log_dir=f"output/hpobench/{task_name}/{i}/random",
    #     verbose=1,
    # ).search(max_evals=max_evals)

    # # BO
    # print("Bayesian optimization...")
    # for scaler, strategy in itertools.product(scalers, strategies):
    #     result = CBO(
    #         hpo.problem,
    #         hpo.run,
    #         surrogate_model="RF",
    #         objective_scaler=scaler,
    #         moo_scalarization_strategy=strategy,
    #         log_dir=f"output/hpobench/{task_name}/{i}/d-{strategy.lower()}-{scaler}",
    #         verbose=1,
    #     ).search(max_evals=max_evals)

    # NSGAII
    # print("NSGAII...")
    # result = MPIDistributedOptuna(
    #     hpo.problem,
    #     hpo.run,
    #     sampler="NSGAII",
    #     n_objectives=2,
    #     log_dir=f"output/hpobench/{task_name}/{i}/nsgaii",
    #     verbose=1,
    # ).search(max_evals=max_evals)

    # MoTPE
    # print("MoTPE...")
    # result = MPIDistributedOptuna(
    #     hpo.problem,
    #     hpo.run,
    #     sampler="TPE",
    #     n_objectives=2,
    #     log_dir=f"output/hpobench/{task_name}/{i}/tpe",
    #     verbose=0,
    # ).search(max_evals=max_evals)

    print("Bayesian optimization...")
    result = CBO(
        hpo.problem,
        hpo.run,
        surrogate_model="RF",
        objective_scaler="quantile-uniform",
        moo_scalarization_strategy="Chebyshev",
        log_dir=f"output/hpobench/{task_name}/{i}/dirichlet",
        verbose=1,
    ).search(max_evals=max_evals)

Starting round 0...
Bayesian optimization...


/Users/romainegele/Documents/Argonne/deephyper/deephyper/evaluator/_evaluator.py:127: UserWarning: Applying nest-asyncio patch for IPython Shell!
  warnings.warn(


  0%|          | 0/200 [00:00<?, ?it/s]

Starting round 1...
Bayesian optimization...


  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

Starting round 8...
Bayesian optimization...


  0%|          | 0/200 [00:00<?, ?it/s]

Starting round 9...
Bayesian optimization...


  0%|          | 0/200 [00:00<?, ?it/s]